In [9]:
import os
import re
import numpy

import pandas
import colorcet

import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()

Loading BokehJS ...

In [10]:
dfs = []

regex = re.compile("(.+)_lac.csv")

for file in os.listdir("data"):
    
    match = regex.search(file)
    
    if match is None:
        continue
    
    genotype = match.group(1)
    
    df = pandas.read_csv(os.path.join("data", file), comment="#")
    
    df["genotype"] = genotype
    
    dfs.append(df)
    
df = dfs[0].append(dfs[1:])

In [11]:
df

,[IPTG] (mM),fold change,genotype
0,0.000009,0.023387,q18m
1,0.001003,0.033297,q18m
2,0.002006,0.033669,q18m
3,0.003907,0.038964,q18m
4,0.008008,0.044287,q18m
...,...,...,...
15,1.480551,0.723629,wt
16,3.108100,0.748718,wt
17,6.044332,0.768828,wt
18,11.494881,0.754358,wt


In [12]:
def scatter(data, cat, x, y, x_axis_log_scale=False):
    
    figure = bokeh.plotting.figure(
        x_axis_label=x,
        y_axis_label=y,
        x_axis_type="log" if x_axis_log_scale else None
    )
    
    colors = colorcet.palette["glasbey_category10"]
    
    for i, category in enumerate(data[cat].unique()):
        
        category_mask = data[cat] == category
    
        figure.circle(
            source=data[category_mask],
            x=x,
            y=y,
            color=colors[i],
            legend_label=category
        )
    
    bokeh.io.show(figure)

In [13]:
scatter(df, "genotype", df.columns[0], df.columns[1], x_axis_log_scale=True)

In [14]:
mutant_RK_values = {
    "q18m": 1332,
    "q18a": 16.56,
    "wt": 141.5
}

In [15]:
def fold_change(c, RK, KdA=0.017, KdI=0.002, Kswitch=5.8):
    """Take an array of IPTG values and compute the fold change"""
    
    fc = RK * numpy.power(1 + c / KdA, 2)
    fc /= numpy.power(1 + c / KdA, 2) + Kswitch * numpy.power(1 + c / KdI, 2)
    fc += 1
    fc = 1 / fc
    
    return fc

In [16]:
X_COLUMN_NAME = "[IPTG] (mM)"
Y_COLUMN_NAME = "fold change"

figure = bokeh.plotting.figure(
    x_axis_label=X_COLUMN_NAME,
    y_axis_label=Y_COLUMN_NAME,
    x_axis_type="log"
)

colors = colorcet.palette["glasbey_category10"]

theoretical_x = numpy.logspace(-6, 1, 100)

for i, genotype in enumerate(df["genotype"].unique()):

    genotype_mask = df["genotype"] == genotype

    figure.circle(
        source=df[genotype_mask],
        x=X_COLUMN_NAME,
        y=Y_COLUMN_NAME,
        color=colors[i],
        legend_label=genotype
    )
    
    theoretical_y = fold_change(theoretical_x, mutant_RK_values[genotype])
    
    figure.line(
        x=theoretical_x,
        y=theoretical_y,
        color=colors[i],
        legend_label="%s theoretical" % genotype
    )

bokeh.io.show(figure)